In [ ]:
%matplotlib inline

# Autograd: 自动微分

PyTorch 中所有神经网络的核心是 `autograd` 软件包。
让我们先简单了解一下这个包，然后去训练我们的第一个神经网络。

`autograd` 软件包为 Tensor 上的所有操作提供自动微分。
它是一个 define-by-run 的框架，这意味着你的反向传播由你的代码的运行方式定义，并且每次迭代都可以不同。

让我们用更简单的描述和一些例子来了解这一特性吧。

## Tensor

`torch.Tensor` 是 PyTorch 软件包的核心类。
如果你把 Tensor 的属性 `.requires_grad` 设置为 `True`，它就会开始跟踪所有在它身上发生的操作。
当你结束计算的时候，你可以调用 `.backward()` 方法，这时所有的梯度都会自动计算好。
这个 Tensor 的梯度将会累积到它的 `.grad` 属性中。

想要阻止一个 Tensor 跟踪历史，你可以调用 `.detach()` 方法将它从计算历史中分离出来，并阻止它在以后的计算中被跟踪。

要阻止跟踪历史（和使用内存），你还可以使用 `with torch.no_grad():` 包裹代码块。
这可能在 evaluate 一个模型的时候非常管用，因为模型可能含有不需要计算梯度的包含 `requires_grad=True` 属性的可训练参数。

还有一个对于 autograd 的实现非常重要的类 - `Function` 。

`Tensor` 和 `Function` 互相连接并构建一个编码了完整计算历史的无环图。
每一个 Tensor 都有一个 `.grad_fn` 属性，该属性包含创建这个 `Tensor` 的 `Function` 的引用（除了 user 创建的 Tensor ，因为它们的 `grad_fn` 是 `None` ）。

如果你想要计算导数，你可以在一个 `Tensor` 上调用 `.backward()` 方法。
如果 `Tensor` 是一个标量（即它只有一个数据元素），你不需要为 `.backward()` 指定任何参数，但是如果它有更多元素，你需要指定一个形状匹配的张量作为 `gradient` 参数。


In [ ]:
import torch

创建一个 Tensor 并设置属性 `requires_grad=True` 以跟踪计算


In [ ]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

进行 Tensor 操作：


In [ ]:
y = x + 2
print(y)

`y` 是一个操作的结果，所以它具有 `grad_fn` 属性。


In [ ]:
print(y.grad_fn)

在 `y` 上进行更多操作


In [ ]:
z = y * y * 3
out = z.mean()

print(z, out)

`.requires_grad_( ... )` 更改一个现有的 Tensor 的 `requires_grad` 属性。
如果没有给出， `requires_grad` 属性的默认值是 `False` 。


In [ ]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

## 梯度（Gradients）
现在让我们进行反向传播吧。因为 `out` 只包含一个标量，所以 `out.backward()` 等价于 `out.backward(torch.tensor(1.))` 。


In [ ]:
out.backward()

打印梯度 $d(out) / dx$


In [ ]:
print(x.grad)

你应该得到了一个全是 `4.5` 的矩阵。我们先把 `out` 记作 *Tensor* “$o$” 。

我们已经知道 $o = \frac{1}{4}\sum_i z_i$ ， $z_i = 3(x_i+2)^2$ 和 $z_i\bigr\rvert_{x_i=1} = 27$ 。

所以 $\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$ ，

因此 $\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$ 。


在数学上，如果你有一个向量值函数 $\vec{y}=f(\vec{x})$ ，那么 $\vec{y}$ 关于 $\vec{x}$ 的梯度是一个雅可比矩阵（Jacobian matrix）：

\begin{align}J=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\end{align}

一般来说， `torch.autograd` 是一个计算雅可比向量积的引擎。
也就是说，给定任意向量$v=\left(\begin{array}{cccc} v_{1} & v_{2} & \cdots & v_{m}\end{array}\right)^{T}$ ， 计算向量积 $J\cdot v$ 。 
如果 $v$ 恰好是标量函数 $l=g\left(\vec{y}\right)$ 的梯度，也就是说， $v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$ ，那么根据链式法则，雅可比向量积将等于 $l$ 关于 $\vec{x}$ 的梯度：

\begin{align}J\cdot v=\left(\begin{array}{ccc}
   \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
   \vdots & \ddots & \vdots\\
   \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
   \end{array}\right)\left(\begin{array}{c}
   \frac{\partial l}{\partial y_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial y_{m}}
   \end{array}\right)=\left(\begin{array}{c}
   \frac{\partial l}{\partial x_{1}}\\
   \vdots\\
   \frac{\partial l}{\partial x_{n}}
   \end{array}\right)\end{align}

雅可比向量积的这一特性使得将外部梯度馈送到一个具有非标量输出的模型中非常方便。


现在让我们看一个雅可比向量积的例子吧：


In [ ]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

在这个实例中， `y` 不再是一个标量了。 `torch.autograd` 无法直接计算完整的雅可比矩阵，但是如果我们只想得到雅可比向量积，则只需要将向量作为参数传递给 `.backward()` 方法：


In [ ]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

你也可以使用 `with torch.no_grad():` 包裹代码块来停止 autograd 跟踪在属性为 `.requires_grad=True` 的 Tensor 上的历史：


In [ ]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

**稍后阅读：**

　　``autograd`` 和 ``Function`` 的文档在 https://pytorch.org/docs/autograd
